In [23]:
import pandas as pd
import nltk

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

Обучите логистическую регрессию так, чтобы она определяла тональность текста.  
Подсчитайте величину TF-IDF для текстов. Лемматизированные тексты твитов для обучения находятся в файле tweets_lemm_train.csv. Целевой признак вы найдёте в столбце positive.  
  
Обученной моделью классификации определите результаты предсказания для тестовой выборки твитов, которая лежит в файле tweets_lemm_test.csv. В этой выборке целевого признака нет. Сохраните предсказания в столбце positive. Таблицу с результатом сохраните как csv-файл, но чтобы тренажёр принял файл, не указывайте расширение (например, назовите файл 'predictions')  
  
Значение accuracy вашей модели должно быть не меньше 0.62.

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /home/bionic/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
tweets_lemm_train = pd.read_csv(
        './datasets/tweets_lemm_train.csv',
        usecols=['positive', 'lemm_text']
    )

tweets_lemm_test= pd.read_csv(
        './datasets/tweets_lemm_test.csv',
        usecols=['lemm_text']
    )

In [32]:
print(
    len(tweets_lemm_train.loc[tweets_lemm_train['positive'] == 1].index),
    len(tweets_lemm_train.loc[tweets_lemm_train['positive'] == 0].index),
    len(tweets_lemm_train.index),
    
)

2500 2500 5000


In [33]:
display(tweets_lemm_train.head(2))
display(tweets_lemm_test.head(3))

,positive,lemm_text
0,1,хоть я и школотый но поверь у мы то же самый о...
1,1,да весь таки он немного похожий на он но мой м...


,lemm_text
0,если крис так интересовать ребёнок то либо они...
1,по рубль в месяц можно разместить ссылка на те...
2,учитывать что сейчас преобладать один половина...


In [35]:
features_tweets_lemm_train = tweets_lemm_train['lemm_text'].values.astype('U')
features_tweets_lemm_test = tweets_lemm_test['lemm_text'].values.astype('U')

In [36]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf.fit(features_tweets_lemm_train)

tf_idf_train = count_tf_idf.transform(features_tweets_lemm_train)
tf_idf_test = count_tf_idf.transform(features_tweets_lemm_test)

print("Размер матрицы:", tf_idf_train.shape)

Размер матрицы: (5000, 9737)


In [20]:
log_reg = LogisticRegression()
log_reg.fit(tf_idf_train, tweets_lemm_train['positive'])

LogisticRegression()

In [25]:
scores_log_reg = cross_val_score(
        log_reg,
        tf_idf_train,
        tweets_lemm_train['positive'],
        scoring='accuracy',
        cv=7
    )

accuracy = 0.60745098
print(accuracy)

In [26]:
print(scores_log_reg)

[0.62797203 0.63216783 0.67086835 0.6442577  0.62745098 0.60644258
 0.60084034]


In [37]:
log_reg_pred = log_reg.predict(tf_idf_test)

In [38]:
display(log_reg_pred)

array([1, 0, 0, ..., 0, 0, 1])

In [48]:
tweets_lemm_test['positive'] = log_reg_pred

In [49]:
display(tweets_lemm_test)

,lemm_text,predictions,positive
0,если крис так интересовать ребёнок то либо они...,1,1
1,по рубль в месяц можно разместить ссылка на те...,0,0
2,учитывать что сейчас преобладать один половина...,0,0
3,товарищ но я никак не мочь отдельно не отметит...,0,0
4,квн быть отличный оооочень понравиться женский...,1,1
...,...,...,...
2995,жуйк ачивка в контра зло мой младший брат втян...,0,0
2996,не хотеть идти на танец,0,0
2997,на улица пусто ни человек ни машина наверно хо...,0,0
2998,по мой мы и ходить за водка в три час за это ч...,0,0


In [50]:
tweets_lemm_test.to_csv(
    path_or_buf='./predictions',
    columns=['positive'],
    # index=False,
    # header=False,
)